In [6]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr # type: ignore

#Data viz
import plotly.graph_objs as go # type: ignore
import plotly.express as px # type: ignore

import time
from datetime import date

## 1) Understanding Data-Driven Decisions data pulls

In [9]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 7; day=17
Period for indexes: 1954-07-17 to 2024-07-17 


## 1.1) GDP

In [10]:
# Real Potential Gross Domestic Product (GDPPOT), Billions of Chained 2012 Dollars, QUARTERLY
# https://fred.stlouisfed.org/series/GDPPOT

gdppot = pdr.DataReader("GDPPOT", "fred", start=start)